In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.2 MB/s eta 0:00:02
   ------------ --------------------------- 0.8/2.5 MB 1.2 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.2 MB/s eta 0:00:02
   --------------------- ------------------ 1.3/2.5 MB 1.2 MB/s et

In [4]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
df.shape

(569, 33)

In [7]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

### Numpy arrays to PyTorch tensors

*   List item
*   List item


In [11]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

### Defining the model

In [12]:
class MySimpleNN():

  def __init__(self, X):

    self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss

### Important Parameters

In [14]:
learning_rate = 0.1
epochs = 25

### Training Pipeline


In [15]:
# create model
model = MySimpleNN(X_train_tensor)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.602947239916057
Epoch: 2, Loss: 3.4686693713357433
Epoch: 3, Loss: 3.3316844332003215
Epoch: 4, Loss: 3.1853838103266656
Epoch: 5, Loss: 3.0342272832569623
Epoch: 6, Loss: 2.8762890026005126
Epoch: 7, Loss: 2.7140644038002972
Epoch: 8, Loss: 2.546563448299683
Epoch: 9, Loss: 2.3766903629980414
Epoch: 10, Loss: 2.2026612929361056
Epoch: 11, Loss: 2.026034939185521
Epoch: 12, Loss: 1.8516624890270756
Epoch: 13, Loss: 1.6815957013982947
Epoch: 14, Loss: 1.5218570006471035
Epoch: 15, Loss: 1.3746944185760128
Epoch: 16, Loss: 1.2425811495780092
Epoch: 17, Loss: 1.1279569569210908
Epoch: 18, Loss: 1.0326942914184853
Epoch: 19, Loss: 0.9572549221423579
Epoch: 20, Loss: 0.9000082743668463
Epoch: 21, Loss: 0.8576416153283101
Epoch: 22, Loss: 0.8264422116586697
Epoch: 23, Loss: 0.8032261829654721
Epoch: 24, Loss: 0.7855968853524746
Epoch: 25, Loss: 0.7718621977412433


In [16]:
model.bias

tensor([-0.1564], dtype=torch.float64, requires_grad=True)

### Evaluation

In [17]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6184980273246765


## defining the model

In [25]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [26]:
learning_rate = 0.1
epochs = 25

In [27]:
# define loss function
loss_function = nn.BCELoss()

In [31]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

X_train_tensor = X_train_tensor.float()
y_train_tensor = y_train_tensor.view(-1, 1).float()

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor)

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7685068249702454
Epoch: 2, Loss: 0.5653722882270813
Epoch: 3, Loss: 0.45869579911231995
Epoch: 4, Loss: 0.3943593502044678
Epoch: 5, Loss: 0.35116833448410034
Epoch: 6, Loss: 0.31998974084854126
Epoch: 7, Loss: 0.29629039764404297
Epoch: 8, Loss: 0.2775702476501465
Epoch: 9, Loss: 0.2623385190963745
Epoch: 10, Loss: 0.24965113401412964
Epoch: 11, Loss: 0.23888090252876282
Epoch: 12, Loss: 0.2295946180820465
Epoch: 13, Loss: 0.221483051776886
Epoch: 14, Loss: 0.2143193781375885
Epoch: 15, Loss: 0.20793326199054718
Epoch: 16, Loss: 0.20219412446022034
Epoch: 17, Loss: 0.19700005650520325
Epoch: 18, Loss: 0.1922702193260193
Epoch: 19, Loss: 0.18793965876102448
Epoch: 20, Loss: 0.18395540118217468
Epoch: 21, Loss: 0.18027392029762268
Epoch: 22, Loss: 0.17685894668102264
Epoch: 23, Loss: 0.17368003726005554
Epoch: 24, Loss: 0.17071139812469482
Epoch: 25, Loss: 0.16793107986450195
